In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from matplotlib.pyplot import plot, ion, show, savefig, cla, figure

## 아래 코드는 colab에서 실행하면 오래 걸려서 데스크탑으로 진행

In [ ]:
# Data_preprocessing.py 파일로 실행
'''

import os
import pandas as pd
import numpy as np

train_path = 'C:/Users/thehb/OneDrive/Desktop/성훈_석사/수업/고급시계열/프로젝트/VAE-LSTM-for-anomaly-detection/datasets/NAB-known-anomaly/csv-files/fall_data/train'
test_path = 'C:/Users/thehb/OneDrive/Desktop/성훈_석사/수업/고급시계열/프로젝트/VAE-LSTM-for-anomaly-detection/datasets/NAB-known-anomaly/csv-files/fall_data/test'

file_list = []
train_file_list = os.listdir(train_path)
test_file_list = os.listdir(test_path)
for file in train_file_list:
    file_list.append(os.path.abspath(train_path+'/'+file))
for file in test_file_list:
    file_list.append(os.path.abspath(test_path+'/'+file))


all_data = []
all_label = []
window_size = 10 # 0.05초
#size = 0
for file_dir in file_list:
    print(file_dir)
    df = pd.read_csv(file_dir, usecols=['gyro_z', 'label'])   
    #size += len(df)
    df.loc[(df.label == 'FOL'), 'label'] = 1
    df.loc[(df.label == 'FKL'), 'label'] = 1
    df.loc[(df.label == 'BSC'), 'label'] = 1
    df.loc[(df.label == 'SDL'), 'label'] = 1
    df.loc[(df.label != 1), 'label'] = 0
    data = np.array(df.gyro_z)
    label = np.array(df.label)
    for i in range(len(data) // window_size):        
        all_data.append(np.mean(data[i*window_size:i*window_size+window_size]))
        all_label.append(0 if np.sum(label[i*window_size:i*window_size+window_size]) < 5 else 1)
#print(size)
len(all_data)

pd.DataFrame(all_data, columns=['values']).to_csv('C:/Users/thehb/OneDrive/Desktop/성훈_석사/수업/고급시계열/프로젝트/data/fall_gyro_z.csv', header= True, index=True)
pd.DataFrame(all_label, columns=['label']).to_csv('C:/Users/thehb/OneDrive/Desktop/성훈_석사/수업/고급시계열/프로젝트/data/fall_label.csv', header= True, index=True)

'''

## Helper functions to load and process original csv files

In [ ]:
# this function load one .cvs (a sequence)
def load_data(csv_folder='/content/drive/MyDrive/Colab Notebooks/Timeseries/Team_project/VAE-LSTM-for-anomaly-detection/datasets/NAB-known-anomaly/csv-files/'):
    '''
    [input]
    dataset: 파일명
    csv_folder: csv 파일이 위치한 폴더명

    [return]
    t: idx 모음
    t_unit: 데이터 간의 시간 간격
    readings: 데이터
    idx_anomaly: anomaly에 해당하는 idx
    '''
    # train/val dataset
    #data_file = os.path.join(csv_folder, 'fall_gyro_z.csv')
    #label_file = os.path.join(csv_folder, 'fall_label.csv')
    
    # test dataset
    data_file = os.path.join(csv_folder, 'test_fall_gyro_z.csv')
    label_file = os.path.join(csv_folder, 'test_fall_label.csv')
    
    t_unit = '0.05 sec'
    
    t = []
    readings = []
    idx_anomaly = []
    with open(data_file) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        print("\n--> Anomalies occur at:")
        for i, row in enumerate(readCSV):
            if i > 0:
                t.append(i)
                readings.append(float(row[1]))
                '''
                for j in range(len(anomalies)):
                    if row[0] == anomalies[j]:
                        idx_anomaly.append(i)
                        print("  timestamp #{}: {}".format(j, row[0]))
                '''
    
    with open(label_file) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for i, row in enumerate(readCSV):
            if i > 0:
                if row[1] == '1':
                    idx_anomaly.append(int(row[0]))
    t = np.asarray(t)
    readings = np.asarray(readings)
    print("\nOriginal csv file contains {} timestamps.".format(t.shape))
    print("Processed time series contain {} readings.".format(readings.shape))
    print("Anomaly indices are {}".format(idx_anomaly))
    
    return t, t_unit, readings, idx_anomaly

In [ ]:
# This function plots a dataset with the train/test split and known anomalies
# Relies on helper function load_data()

def process_and_save_specified_dataset(dataset, idx_split, y_scale=20, save_file=False):
    '''
    [input]
    dataset: csv 파일명
    idx_split: train 데이터에 해당하는 idx 구간(나머지 뒷부분은 test로 사용됨)

    [return]
    t: 전체 데이터 idx
    readings_normalised: 정규화된 readings(데이터)
    '''
    t, t_unit, readings, idx_anomaly = load_data()
    
    # split into training and test sets
    training = readings[idx_split[0]:idx_split[1]]
    t_train = t[idx_split[0]:idx_split[1]]
    
    # normalise by training mean and std 
    train_m = np.mean(training)
    train_std = np.std(training)
    print("\nTraining set mean is {}".format(train_m))
    print("Training set std is {}".format(train_std))
    readings_normalised = (readings - train_m) / train_std
    
    training = readings_normalised[idx_split[0]:idx_split[1]]
    if idx_split[0] == 0:
        test = readings_normalised[idx_split[1]:]
        t_test = t[idx_split[1]:] - idx_split[1]
        idx_anomaly_test = np.asarray(idx_anomaly) - idx_split[1] # anomaly data는 test에만 있다고 정의했나봄
    else:
        test = [readings_normalised[:idx_split[0]], readings_normalised[idx_split[1]:]]
        t_test = [t[:idx_split[0]], t[idx_split[1]:] - idx_split[1]]
        idx_anomaly_split = np.squeeze(np.argwhere(np.asarray(idx_anomaly)>idx_split[0]))
        idx_anomaly_test = [np.asarray(idx_anomaly[:idx_anomaly_split[0]]), 
                            np.asarray(idx_anomaly[idx_anomaly_split[0]:]) - idx_split[1]]
    print("Anomaly indices in the test set are {}".format(idx_anomaly_test))
    
    if save_file:
        save_dir = '/content/drive/MyDrive/Colab Notebooks/Timeseries/Team_project/VAE-LSTM-for-anomaly-detection/datasets/NAB-known-anomaly/'
        np.savez(save_dir+dataset+'.npz', t=t, t_unit=t_unit, readings=readings, idx_anomaly=idx_anomaly,
                    idx_split=idx_split, training=training, test=test, train_m=train_m, train_std=train_std,
                    t_train=t_train, t_test=t_test, idx_anomaly_test=idx_anomaly_test)
        print("\nProcessed time series are saved at {}".format(save_dir+dataset+'.npz'))
    else:
        print("\nProcessed time series are not saved.")

In [ ]:
idx_split = [0,77875] # train(77875개)/val(18350개)

#process_and_save_specified_dataset('fall_data', idx_split, y_scale=4, save_file=True) # train/val
process_and_save_specified_dataset('test_fall_data', idx_split, y_scale=4, save_file=True) # test(총 18352개)


--> Anomalies occur at:

Original csv file contains (90040,) timestamps.
Processed time series contain (90040,) readings.
Anomaly indices are [83875, 83876, 83877, 83878, 83879, 83880, 83881, 83882, 83883, 83884, 83885, 83886, 83887, 83888, 83889, 83890, 83891, 83892, 83893, 83894, 83895, 83896, 83897, 83898, 83899, 83900, 83901, 83902, 83903, 83904, 83905, 83906, 83907, 83908, 83909, 83910, 83911, 83912, 83913, 83914, 83915, 83916, 83917, 83918, 83919, 83920, 83921, 83922, 83923, 83924, 83925, 83926, 83927, 83928, 83929, 83930]

Training set mean is 0.008316448094505501
Training set std is 0.026930146741187475
Anomaly indices in the test set are [6000 6001 6002 6003 6004 6005 6006 6007 6008 6009 6010 6011 6012 6013
 6014 6015 6016 6017 6018 6019 6020 6021 6022 6023 6024 6025 6026 6027
 6028 6029 6030 6031 6032 6033 6034 6035 6036 6037 6038 6039 6040 6041
 6042 6043 6044 6045 6046 6047 6048 6049 6050 6051 6052 6053 6054 6055]

Processed time series are saved at /content/drive/MyDrive/